In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

In [2]:
s = UNITS.second
N = UNITS.newton
kg = UNITS.kilogram
m = UNITS.meter

<Unit('meter')>

In [65]:
s1x = 90e9
s1y = 0
s1vx = 0 
s1vy = 9000

s2x = -90e9
s2y = 0
s2vx = 0
s2vy = -9000

init = State(s1x = s1x * m,
             s1y = s1y * m,
             s1vx = s1vx * m/s,
             s1vy = s1vy *m/s,
             
             s2x = s2x * m,
             s2y = s2y * m,
             s2vx = s2vx * m/s,
             s2vy = s2vy *m/s,
             )

,values
s1x,90000000000.0 meter
s1y,0 meter
s1vx,0.0 meter / second
s1vy,9000.0 meter / second
s2x,-90000000000.0 meter
s2y,0 meter
s2vx,0.0 meter / second
s2vy,-9000.0 meter / second


In [ ]:
system = System(init=init,
                G=6.674e-11 * N / kg**2 * m**2,
                s1m=1.989e30 * kg,
                s2m=1.989e30 * kg,
                t_0=0 * s,
                t_end=1.5e10* s)

In [ ]:
def universal_gravitation_s1(state, system):
    s1x, s1y, s1vx, s1vy, s2x, s2y, s2vx, s2vy = state
    unpack(system)
    
    #vectors
    orgin_to_s1 = Vector(s1x,s1y)
    orgin_to_s2 = Vector(s2x,s2y)
    
    #direction
    s1_to_s2 = -orgin_to_s1 + orgin_to_s2
    
    s1_to_s2_dir = s1_to_s2.hat()
   
    
    #magnitude
    r = sqrt(((s2x-s1x)**2) + ((s2y-s1y)**2))
        
    mag = G * s1m * s2m / r**2
    
    grav_force_s1= s1_to_s2_dir * mag
    
    return grav_force_s1

In [ ]:
def universal_gravitation_s2(state, system):
    s1x, s1y, s1vx, s1vy, s2x, s2y, s2vx, s2vy = state
    unpack(system)
    
    #vectors
    orgin_to_s1 = Vector(s1x,s1y)
    orgin_to_s2 = Vector(s2x,s2y)
    
    #direction
    s2_to_s1 = -orgin_to_s2 + orgin_to_s1
    
    s2_to_s1_dir =  s2_to_s1.hat()
    
    #magnitude
    r = sqrt(((s2x-s1x)**2) + ((s2y-s1y)**2))
        
    mag = G * s1m * s2m / r**2
    
    #force vector
    grav_force_s2 = s2_to_s1_dir * mag 
    
    return grav_force_s2

In [ ]:
def slope_function(state, t, system):
    s1x, s1y, s1vx, s1vy, s2x, s2y, s2vx, s2vy = state
    unpack(system)
    
    #v = Vector(vx,vy)
    
    s1_a_grav = universal_gravitation_s1(state, system)/s1m
    s2_a_grav = universal_gravitation_s2(state, system)/s2m
    
    
    s1a = s1_a_grav
    s2a = s2_a_grav
    
    return s1vx, s1vy, s1a.x, s1a.y, s2vx, s2vy, s2a.x, s2a.y

In [ ]:
results, details = run_ode_solver(system, slope_function)
details

In [ ]:
day = results.index/86400

def convert_m(meters):
    
    #meters to kilometers
    km = meters/1000
    
    #kilometers to million kilometers
    Mkm = km/1e6
    
    return Mkm

In [ ]:
#Zero Initial Velocity
plot(day,convert_m(results.s1x), label = 'S1: X postion')
plot(day,convert_m(results.s1y), label = 'S1: Y positon')

plot(day,convert_m(results.s2x), label = 'S2: X postion')
plot(day,convert_m(results.s2y), label = 'S2: Y positon')
decorate(title = 'Intitial Velocity = 0m/s',
         xlabel='Time (days)',
         ylabel='Distance from sun (million km)')

In [ ]:
plot(convert_m(results.s1x),convert_m(results.s1y), label ='Sun 1\'s Path')
plot(convert_m(results.s2x),convert_m(results.s2y), label ='Sun 2\'s Path')
decorate(title = 'Intitial Velocity = -30,300 m/s',
         xlabel='Distance from sun (million km)',
         ylabel='Distance from sun (million km)')